In [2]:
import os
import re


inputPath = 'Data'
nonForamtedSentences = []
for filename in os.listdir(inputPath):
    f = open(inputPath + '\\' + filename, 'r',encoding = 'utf-8')
    content = f.read()
    list_ar = content.split(sep ='\n')
    nonForamtedSentences.append(list_ar)


In [1]:
arabicStop = []
f = open('ArabicStop.txt', 'r',encoding = 'utf-8')
content = f.read()
arabicStop = content.split(sep ='\n')

In [12]:
training_corpus = []
for sentence in nonForamtedSentences:
    text = ' '.join(sentence)
    text = re.sub(r'[^\u0627-\u064a\s]','', text)
    text = text.split()
    meaningful_words = [w for w in text if not w in arabicStop]
    training_corpus.append(meaningful_words)

In [11]:
ts =  ' '.join(nonForamtedSentences[0])
ts = re.sub(r'[^\u0627-\u064a\s]','', text)
ts = ts.split()
meaningful_words = [w for w in ts if not w in arabicStop]
meaningful_words

['لندن',
 'بلغ',
 'مانشستر',
 'يونايتد',
 'الدور',
 'نصف',
 'النهاي',
 'مسابقة',
 'كس',
 'رابطة',
 'الاندية',
 'الانكليزية',
 'المحترفة',
 'بفوزه',
 'ارسنال',
 'المباراة',
 'اقيمت',
 'بينهما',
 'ملعب',
 'اولدترافورد',
 'متفرجين',
 'وخاض',
 'ارسنال',
 'المباراة',
 'بتشكيلة',
 'الصف',
 'الرديف',
 'وبلغ',
 'معدل',
 'اعمار',
 'لاعبيه',
 'اشرك',
 'مانشستر',
 'يونايتد',
 'لاعبيه',
 'الاحتياطيين',
 'بينهم',
 'دوليين',
 'ابرزهم',
 'البرازيلي',
 'كليبرسون',
 'الفاز',
 'منتخب',
 'بلاده',
 'بكس',
 'العالم',
 'وفيليب',
 'نيفيل',
 'ووس',
 'براون',
 'تمض',
 'ثانية',
 'نجح',
 'مانشستر',
 'يونايتد',
 'افتتاح',
 'التسجيل',
 'بواسطة',
 'مهاجمه',
 'الفرنسي',
 'دافيد',
 'بيليون',
 'تخطى',
 'مدافعين',
 'يسدد',
 'مشارف',
 'المنطقة',
 'كرة',
 'قوية',
 'افلتت',
 'يدي',
 'الحارس',
 'الاسباني',
 'مانويل',
 'المونيا',
 'وتهادت',
 'داخل',
 'شباكه',
 'وسيطر',
 'مانشستر',
 'يونايتد',
 'مجريات',
 'اللعب',
 'وسط',
 'الملعب',
 'الشوط',
 'وسنحت',
 'لاعبه',
 'كيرون',
 'ريتشاردسون',
 'فرصة',
 'لتعزيز',
 'تقدم',
 'فريقه',

In [13]:
numberOfWords = 0
for sentence in training_corpus:
    numberOfWords += len(sentence)
print(numberOfWords)



573053


In [14]:
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

E:\Anaconda\envs\NoPytorch\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [15]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(training_corpus, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[training_corpus], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)



E:\Anaconda\envs\NoPytorch\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [16]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [17]:
# Form Bigrams
data_words_bigrams = make_bigrams(training_corpus)

In [18]:
# Create Dictionary
id2word = corpora.Dictionary(training_corpus)

# Create Corpus
texts = training_corpus

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]



In [21]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [22]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.006*"بالماة" + 0.005*"بلغت" + 0.005*"عماني" + 0.004*"سهم" + 0.004*"نقطة" '
  '+ 0.004*"سعر" + 0.004*"العام" + 0.004*"بنسبة" + 0.004*"بلغ" + '
  '0.004*"اسعار"'),
 (1,
  '0.011*"المركز" + 0.009*"محمد" + 0.007*"البطولة" + 0.007*"سعيد" + '
  '0.007*"مسابقة" + 0.006*"ريس" + 0.006*"الرياضية" + 0.006*"عبدالله" + '
  '0.005*"القوى" + 0.005*"الشيخ"'),
 (2,
  '0.016*"الاولمبية" + 0.008*"الالعاب" + 0.007*"اثينا" + 0.007*"النهاي" + '
  '0.007*"العالم" + 0.007*"دورة" + 0.005*"ثينا" + 0.005*"سباق" + 0.004*"ذهبية" '
  '+ 0.004*"الدورة"'),
 (3,
  '0.006*"العربية" + 0.004*"لى" + 0.004*"الدول" + 0.004*"السلطنة" + '
  '0.004*"العمل" + 0.003*"دول" + 0.003*"كلية" + 0.002*"الدورة" + 0.002*"وذلك" '
  '+ 0.002*"القطاع"'),
 (4,
  '0.012*"المباراة" + 0.008*"المنتخب" + 0.007*"منتخبنا" + 0.006*"الشوط" + '
  '0.006*"فريق" + 0.006*"الفريق" + 0.005*"الدور" + 0.005*"القدم" + '
  '0.005*"الفوز" + 0.005*"مباراة"')]


In [23]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      41.264977        1       1  0.122825 -0.066448
2      20.700779        1       2  0.130454  0.197605
4      16.623398        1       3 -0.286592  0.115154
1      15.160580        1       4 -0.073474 -0.231264
0       6.250274        1       5  0.106787 -0.015047, topic_info=      Category         Freq       Term        Total  loglift  logprob
term                                                                 
21994  Default  2027.000000  الاولمبية  2027.000000  30.0000  30.0000
54162  Default  1147.000000   المباراة  1147.000000  29.0000  29.0000
40237  Default  1001.000000    الالعاب  1001.000000  28.0000  28.0000
46175  Default  1188.000000     المركز  1188.000000  27.0000  27.0000
49137  Default   804.000000    المنتخب   804.000000  26.0000  26.0000
31291  Default   882.000000      اثينا   882.000000  25.0000  25.0000
40190  Default   991.000000       محمد   991.000000  24.0000  24.0000
44118  Default   888.000000     النهاي   888.000000  23.0000  23.0000
32674  Default  1081.000000     العالم  1081.000000  22.0000  22.0000
62391  Default   931.000000    البطولة   931.000000  21.0000  21.0000
54629  Default   932.000000       دورة   932.000000  20.0000  20.0000
466    Default   812.000000    منتخبنا   812.000000  19.0000  19.0000
38880  Default   606.000000       سعيد   606.000000  18.0000  18.0000
9658   Default   696.000000     مسابقة   696.000000  17.0000  17.0000
37185  Default   771.000000      القوى   771.000000  16.0000  16.0000
42723  Default   588.000000      الشوط   588.000000  15.0000  15.0000
42828  Default   756.000000       فريق   756.000000  14.0000  14.0000
13357  Default   482.000000    عبدالله   482.000000  13.0000  13.0000
45014  Default   500.000000   الرياضية   500.000000  12.0000  12.0000
54040  Default   654.000000       ثينا   654.000000  11.0000  11.0000
54950  Default   478.000000      الفوز   478.000000  10.0000  10.0000
48920  Default   880.000000        ريس   880.000000   9.0000   9.0000
36605  Default   597.000000     الفريق   597.000000   8.0000   8.0000
53408  Default   465.000000      الشيخ   465.000000   7.0000   7.0000
44672  Default   463.000000     مباراة   463.000000   6.0000   6.0000
38895  Default   462.000000       لكرة   462.000000   5.0000   5.0000
23547  Default   437.000000       سالم   437.000000   4.0000   4.0000
229    Default   543.000000      الدور   543.000000   3.0000   3.0000
14019  Default   523.000000      القدم   523.000000   2.0000   2.0000
56675  Default  1617.000000    العربية  1617.000000   1.0000   1.0000
...        ...          ...        ...          ...      ...      ...
7873    Topic5    52.983337       زغرب    54.092369   2.7518  -6.5162
20145   Topic5    52.983337   بارتيزان    54.092369   2.7518  -6.5162
57468   Topic5    52.983337    بيفيرين    54.092369   2.7518  -6.5162
15453   Topic5    44.133698     بالسوق    45.095284   2.7510  -6.6990
41513   Topic5    43.173332     ارتفعت    44.134163   2.7505  -6.7210
3227    Topic5    41.724220        مشر    42.674957   2.7500  -6.7551
50112   Topic5    41.851051     اليورو    42.804897   2.7500  -6.7521
62435   Topic5    45.701748      المشر    46.773178   2.7494  -6.6641
17061   Topic5    60.406780       مجان    61.863232   2.7487  -6.3851
2196    Topic5    52.554935   وواتفورد    53.819462   2.7488  -6.5243
42447   Topic5    56.145283    توتنهام    57.615326   2.7467  -6.4582
53376   Topic5   174.622452      عماني   196.389130   2.6551  -5.3236
14047   Topic5    49.087223      وبلغت    50.527203   2.7436  -6.5926
32423   Topic5   216.983704     بالماة   269.568787   2.5555  -5.1064
24310   Topic5   143.698563      بنسبة   178.371414   2.5564  -5.5185
53400   Topic5   183.675934       بلغت   241.353424   2.4995  -5.2730
62567   Topic5   110.468254     مقارنة   138.323730   2.5477  -5.7815
36877   Topic5   141.180115        بلغ   217.803345   2.3390  -5.5362
36029   T